# 微鲤看看之推荐系统实践

presented by 林晟

## 1. 推荐演进思路 及 v3 架构

从多策略人工融合 到 机器聚合 

- `召回阶段 Recall`: (百万/千万 -> 几百)
    - 召回阶段的目的是用快速, 较简单的模型来筛选可能的候选物品
- `排序阶段 Ranking`: 推荐应用的部分 (几百 -> 十几)
    - 排序阶段的目的是用更精确复杂的模型, 进一步对候选物品进行排序, 排序的分数可以是单纯的预估点击率, 也可以是融入业务需求的加权分数

> 分为两个阶段的主要原因是, 在大数据量下, 当前的端到端方案无法满足推荐服务的速度要求. 



<img src="images/dl_in_rec_sys.png" width="700"/>

<img src="images/ssy/recommendation_arch_2019.png" width="700"/>

<img src="images/ssy/ulike2.png" width="700"/>

## 2. 召回阶段 Recall

尽可能从较为丰富的维度, 对用户可能感兴趣的条目进行预筛选

- `ALS`: 从矩阵分解的角度, 用纯机器学习的方法, 求出近似的对用户和物品的表示, 从而进行推荐
 
- `ItemBased`: 以所有用户对每个条目的点击行为为向量, 对两两物品进行相似度的计算, 从而给用户推荐他点过内容相似的内容

- `Item2Vec` : 从用户点击物品的序列, 通过 word2vec 的方法, 计算物品的向量表示, 从而计算物品相似度
 
- `Userprofile`: 对用户进行长期的画像培养, 通过用户的标签和内容标签的匹配程度, 来进行推荐
 
- `Topic`: 分析每一个内容的语义信息和主题分布, 计算用户历史看过的内容的主题分布, 选出主题最为吻合的, 进行推荐

- `Hot`: 热榜相关, e.g. 全局热榜, 频道热榜

### 2.1. ALS Implicit Collaborative Filtering 隐因子召回

**矩阵分解**

这里其实我们是把每一个用户都用一个向量进行表示, 每一个物品也用一个向量来表示; 而向量的长度, 是我们选择的隐因子的数量. ALS 是矩阵分解的其中一种实现.

<img src="images/ssy/als.png" width="700"/>

**原理 & 直觉**

- 如果我们可以将评分矩阵分解出来, 并且两矩阵相乘能够近似的还原原始矩阵; 那么用户向量与物品向量的点乘就是预测的分数了.

**计算**

- 最近48小时内的数据

**显式反馈和隐式反馈 (Explicit and Implicit data)**

不同场景

- 显式反馈数据: 明确表示用户喜好的分数, 例如对音乐, 电影, 书籍等产品的打分
    - 优点: 对每一个用户的喜好表达很明确
    - 缺点: 数据少且不容易获得, 只适合特定业务场景
    

- 隐式反馈数据: 用户的各种行为, 例如付费行为, 某一首歌播放了多少次, 内容点击行为等等
    - 优点: 数据多且容易获得, 适用范围广
    - 缺点: 不明确, 噪声相对多


### 2.2. Itembased 相似条目召回

这里所说的其实是基于物品的协同过滤 (Item-based Collaborative Filtering), item-based 也就是说计算的是物品的相似度, CF 也就是说用用户的行为来计算

<img src="images/ssy/itembased.png" width="550"/>

**原理 & 直觉**

- 每一列表示了一个 item, 通过计算每一列向量的两两相似度, 得出 item 的相似度
- `余弦相似度` : 在信息检索和文本挖掘中广泛使用，在基于物品的推荐方法中，效果也非常好，已经被当作标准的度量体系。

**计算**

- 最近48小时数据
- $ Score = 3*{Count}_{click} - 10*{Count}_{dislike}$
- Ulike实践, 为每一个条目准备它的一批最相似的内容 (输出 Top N), 在实时计算中, 很快的给用户推荐刚才点过相似的内容. 响应的速读最快, 通常下一次刷新 feed 的时候就可以体现出来

<img src="images/ssy/vector_space.png" width="550"/>

**ItemBased IUF**

现在我们知道每个用户的行为都会对物品的相似度计算做出贡献, 上面的算法把所有人都一视同仁了; 但考虑这样一个场景, 我们有的用户是专注与阅读的, 有的用户是专注于赚金币的, 可能会无规律的点击很多的内容条目, 那么我们是不是需要降低他在相似度计算时的

- $IUF = log(\frac{C_{I\_total}}{C_{I\_u}\:\: + 1})$

- $\hat{Score} = Score * IUF$


$C_{I\_total}$ 是参与计算的物品总数, $C_{I\_u}$ 是一个用户有交互的物品总数

当然上面的方法只是对过于活跃的用户做一个软性的惩罚, 对于超级活跃或者机刷行为, 我们可以直接忽略/剔除这个用户.

**看几个例子**

812924894, 吃了中国的油条，再看看印度炸油条过程，网友：能吃下去的是牛人

    ('718694800', 0.16090, 最“坑人”的4款网红零食，脏脏包、紫皮糖上榜，网友：全吃过), 
    ('741532159', 0.14893, 减肥坚持吃这5种食物，说不定能帮你瘦成闪电), 
    ('790487452', 0.10731, 卧室软装搭配——床头柜也有好多样), 
    ('887390189', 0.10641, 不放水，不放面粉，三天两头做一次，酥软香甜，刚出锅我连吃3个), 
    ('785806234', 0.10011, 红薯鸡蛋早餐饼，不用和面，简单快手，孩子都爱吃), 
    ('812962899', 0.09783, 开学了，多给孩子喝这汤，鲜甜味美，保护肠胃), 
    ('808615967', 0.09667, 秋天胃口好！教你9道家常菜，4荤4素1道汤，美味又好吃，学起来), 
    ('793646411', 0.09545, 国内佛系土豪吃龙虾，一桌子龙虾，默不作声几分钟就吃完了), 
    ('883935682', 0.09249, 推荐两道冷门的家常菜做法，不同做法，却是同样的美味), 
    ('884358015', 0.09238, 7种“戒不掉”的小吃，能戒掉一种给你100元，网友：钱拿走！)
 
893087257, 一旦中国爆发战争，后果如何？俄专家给出答案，出乎所有人意料

    ('861124658', 0.15957, 18岁被张艺谋看中，20岁为他生下孩子，因超生被罚748万), 
    ('893087255', 0.15069, 刚刚，英国就南海问题表态了，这次能否说话算数？), 
    ('892572722', 0.13894, 日本自卫队为招兵提高女兵比例，真实目的你可能想象不到！), 
    ('889924671', 0.12819, 路人街拍：凹凸身材的高挑女孩，随便一穿，整个的气质不一样了), 
    ('893087229', 0.12289, 世界最出名的雇佣兵组织，只要钱不要命), 
    ('892572720', 0.11688, 究竟发生了什么？美F22紧急起飞，白宫：一架轰炸机携核弹逼近), 
    ('892572721', 0.11210, 如果不是别人说，还真看不出这是从航母上拆下的，像是在卖废品), 
    ('893087231', 0.10709, 一个被国人误解了近百年的群体，由汉奸组成却能刺杀日本少将), 
    ('893087250', 0.10459, 我军中有黑人士兵? 有的, 连排长也都是黑人), 
    ('893087282', 0.10458, 美军提出无理要求，要派准航母前往 香港访问，回复令人很提气), 
    
894183751, 明星拍雨戏，杨紫自然，舒畅最走心，迪丽热巴堪称教科书模版！

    ('894183745', 0.13351, 秦岚和董洁两版富察皇后哪个更像白月光，机场秀场剧照多图对比), 
    ('892709628', 0.12841, 香港真正的“四大天王”不是刘德华张学友，而是这四位大哥大！), 
    ('890259921', 0.11942, 盘点女童星发展现状，杨紫关晓彤接剧不断，张雪迎靓丽现身), 
    ('894183759', 0.11250, 孙怡晒与于朦胧“互咬”截图：出来混是要还的，粉丝心疼董子健), 
    ('894183748', 0.11206, 宫斗剧的意义是什么？为了告诉所有观众，宫斗比你想象中要残忍), 



**ItemCF 和 UserCF 比较**

**推荐结果**
- UserCF 的推荐结果着重与反应和用户兴趣相似的小群体的热点, 更社会化
- ItemCF 的推荐结果着重于维系用户历史兴趣, 更个性化


**性能适用**
- UserCF 适合用户相对物品较少的
- ItemCF 适合物品相对于用户较少的


**领域适用**
- UserCF 时效性较强, 用户个性化兴趣不明显的领域
- ItemCF 长尾物品丰富, 用户个性化需求强烈的领域

**实时性**
- UserCF 用户的新行为不会立刻反映到推荐结果上
- ItemCF 用户的新行为立刻反应出来


**冷启动**
- UserCF 新用户冷启动差(需要重新计算用户相似度表); 新物品需要等有产生行为之后
- ItemCF 利于新用户冷启动, 有任何点击之后, 立即就能推荐相似内容; 新物品也需要等有产生行为之后


**解释性**
- UserCF 不易解释
- ItemCF 利用历史行为解释

### 2.3. Item2Vec 召回

Item2Vec 借鉴了 NLP 中 Word2Vec 的思想, 从用户交互的物品组成的序列中, 学习到每个物品的向量表示. 再通过相似度的方法进行召回.

<img src="images/ssy/item2vec.png" width="550"/>

### 2.4. Userprofie 用户画像(标签)召回

**离线画像计算: 长期累加 & 时间衰减**

<img src="images/ssy/up.png" width="600"/>

- $S_{d} = S_{d-1}*0.9 + s$

- $S_{d} = S_{d-1}*0.9^1 + S_{d-2}*0.9^2 + S_{d-3}*0.9^3 + ... + S_{1}*0.9^{d-1}$

衰减系数 0.9, $s$ 是当日的标签分数, $S_{d}$是累计到当日的标签分数

**实时画像计算**

因为离线画像是每天计算一次, 所以在计算间隔期间, 我们通过 实时+离线 的方式, 尽量保证用户画像的时效性.

**看个例子**

    show_uid(204638890832)
    =========> level0 tag
    1013694	198.18	娱乐	
    1086   	46.76	历史	
    1513   	42.87	女神	
    1013691	20.94	社会	
    1335   	19.11	热点新闻
    1151146	16.13	小说	
    ...
    ...
    
    =========> level2 tag
    2456307	22.22	全网资讯
    2460336	21.07	唔哩头条
    2059884	14.55	女人	
    2320906	14.15	公公	
    2210329	14.15	红包	
    2319447	13.26	帅气	
    2484983	11.46	环球时报
    2326240	11.46	士兵	
    2319424	11.46	哨兵	
    2386209	11.46	军列	
    2357506	11.46	日丹诺	
    2320156	9.44	粉丝	
    2708502	9.44	孟婆	
    2107083	9.44	演唱会	
    2232132	9.44	杨柳	
    2319414	8.96	饰演	
    2108267	8.54	视频
    ...
    ...


**召回计算**

$r_{u\_i} = q_i * \sum_{i=1}^n (tif_i*c_{i\_t}*s_{u\_t})$

- $q_i$ : 内容质量,平滑 ctr 的平方
- $c_{i_t}$ : 内容c和标签i的关联度, 有就是1
- $s_{u_t}$ : 用户 u 对标签i 的分数
- $tif_i$ : 标签t 的 inverse-log frequency

**原理 & 直觉** 

- 考虑用户标签和条目标签的相关性
- 考虑内容的质量
- 考虑热门标签的降权


### 2.5. Topic 主题召回

**原理 & 直觉** 

- 类似于用标签来表示用户, 这里我们用主题来表示用户; 
- 同样的我们先对内容做分析, 不同于标签关键词的表示, 
- 主题模型可以挖掘出内容隐含的语义信息, 可以进行更细致的表示


**计算**
- 分享内容主题分布
- 通过用户最近看了哪些内容, 总结出用户关心的主题分布; 
- 计算用户主题的概率分布和新内容的主题概率分布的近似程度, 输出 Top N

更多

http://nbviewer.jupyter.org/github/Ringares/Ringares.github.io/blob/master/JupyterNotes/%5BBasic%5D%20LDA%20主题模型.ipynb

### 2.6. 小结

从大的分类来划分的话

ALS, Itembased CF 都属于协同过滤推荐算法, 通过用户行为计算推荐结果

用户画像和主题都属于基于内容的算法, 需要提前对内容进行分析

**协同过滤类算法**

**优点**
- 不需要分析用户和物品特征
- 大部分场景中能够产生足够好的结果

**缺点**
- 冷启动问题(包括用户和物品)
- 流行度偏见（有长尾的时候表现不够好）
- 难于提供解释

**基于内容类算法**

**优点**
- 没有物品冷启动问题
- 不需要历史数据
- 没有流行度偏见，可以推荐有罕见特性的物品
- 可以使用用户内容特性来提供解释

**缺点**
- 物品必须有意义, 可分析, 可表示
- 很难有意外，缺少多样性


## 3. 排序阶段 Ranking

**原理 & 直觉**

- 以前是定死的某个算法结果放在信息流返回一页数据的第几位
- 现在是动态的把用户更可能点击的条目排在信息流的前面
- 
- 直觉上说, 一个推荐算法的结果集也是从相对好到相对坏的, 如果固定位置, 那么展现的机会得不到最大化的利用; 
- 而用 算法B的相对好的结果 动态的替换掉 算法A相对差的结果, 就很自然的可以提升效果

**计算**

- 从各召回列表里拿出候选集
- 预估 CTR, 排序, 取出一页所需数据
- 加入广告, 定投, 冷启动等


这个阶段最重要的事情是 **用合理的特征表示用户, 物品, 场景上下文等**

**从 LR 到 GBT 到 FTRL 的演进**

<img src="images/ssy/lr.png" width="750"/>

**特征表示**

http://git.etouch.cn/gitbucket/ssy_recommend/ulike-doc/blob/master/ulike-feature.md

## 4. 其它补充策略

### 4.1. Bandit 算法

**原理 & 直觉**

- 为解决 Exploitation & Exploration 问题
- 帮助用户探索和发现内容 (可能是他自己之前没意识到的), 以及解决新用户冷启动问题; 
- 给用户尝试一类内容, 如果用户喜欢(点击了), 就给他更多这类的内容; 

可以抽象成多臂老虎机问题 (问如何选择来达到最大收益) 来考虑, 尝试多个摇臂, 哪个摇臂奖励多, 越倾向于掰这个摇臂.

**计算**

- Ulike实践, 是使用汤普森采样 (Thompson sampling), 基于贝叶斯思想，全部用概率分布来表达不确定性; 
- 用于给用户选择频道 (老虎机的摇臂就类比成频道), 每个用户, 对每个频道都会生成一个概率, 选择概率最高的那个频道. 
- 而这个概率会由用户过去对 Bandit 推出内容的反馈而决定. 也就是刚才所说的, 用户越喜欢的频道, 被选中的概率就越高. 
- 确定完频道后, 优先取单独的频道热榜进行推荐

### 4.2. 时效热榜

<img src="images/ssy/eg1.jpeg" width="400"/>


**原理 & 直觉**

- 基于全局的热榜, 可以反应大部分人关心的热点, 或者大众口味的内容

**计算**

- 选取48小时内创建的时效条目, 及48小时内的 PV 数据
- 在满足一定阙值的条件下, 输出按 CTR 排序的时效条目热榜

**优点**
- 相对容易实现
- 良好的基准算法
- 有助于解决新用户冷启动问题

**缺点**
- 不会推荐新条目(物品冷启动问题)
- 推荐列表变化慢

### 4.3. 好友在看

<img src="images/ssy/eg2.jpeg" width="400"/>

**原理 & 直觉**

- 凑热闹
- 爱八卦
- 类群本性

**计算**

- 根据通讯录, 好友等社群关系, 如果一个用户点击了某个条目, 则输出给他的双向社群关系(关注列表和被关注列表)
- 在前端展示时, 会打上好友在看的标签

## 其它相关系统

**内容分析系统 Auto-Tag**

从文章内容, 抽取数关键词和主题

- 关键词分析: 分词 + TFIDF + w2v + 主题词表
- 主题分析: LDA


## 6. Todo

- 更多的特征工程
- 神经网络相关

## The End